# Day 7

link: https://adventofcode.com/2022/day/7

ข้อนี้เป็นการจำลอง File System เราเลยสร้าง `class Dir` ขึ้นมา

In [1]:
class Dir

  attr_accessor  :name, :size, :subdirs, :parent

  def initialize(name)
    @name = name
    @size = 0
    @subdirs = []
  end

  def add_subdir(sd)
    @subdirs << sd
    sd.parent = self
  end

  def update_size(diff)
    @size += diff
    @parent.update_size(diff) if @parent
  end

end

nil

โจทย์สนใจแค่ขนาดของ directory ไม่ได้สนใจไฟล์ เพราะฉะนั้นเราไม่จำเป็นต้องจำลอง File ขึ้นมาจริงๆ เก็บแค่ขนาดของ directory ก็พอ สังเกตว่าเมื่อ `update_size` แล้ว เราต้องเพิ่มขนาดให้ทุกๆ parent folder ขึ้นไปจนถึง root ด้วย

จากนั้นเรา process input เพื่อสร้าง directory structure ขึ้นมา

In [2]:
input = IO.read('data/07.txt')

root = Dir.new('/')
cd = root
commands = input.split("$ ")[1..-1]
commands.each{|cmd|
  lines = cmd.split("\n")
  cmd_args = lines[0].split(' ')
  
  if cmd_args[0] == 'cd'
    case cmd_args[1]
    when '/'
      cd = root
    when '..'
      cd = cd.parent
    else
      cd = cd.subdirs.find{|sd| sd.name == cmd_args[1]}
    end
  end

  if cmd_args[0] == 'ls'
    lines[1..-1].each{|res|
      t,n = res.split(' ')
      if t == 'dir'
        cd.add_subdir(Dir.new(n))
      else
        cd.update_size(t.to_i)
      end
    }
  end
}
nil

## Part 1

โจทย์แรกคือให้หา directory ทั้งหมดที่ขนาดไม่เกิน 100000 

เราก็ traverse tree ซะรอบนึงเพื่อเก็บ directory ทั้งหมดลง array (ตรงนี้เอาไว้ใช้ต่อใน part 2 ด้วย ไม่งั้นเราคำนวณ size รวมจากใน method `visit` เลยก็ได้) แล้วก็ filter เอา directory ที่ต้องการออกมาได้ง่ายๆ

In [3]:
def visit(dir, acc)
  acc << dir
  dir.subdirs.each{|sd| visit(sd, acc)}
end

all_dirs = []
visit(root, all_dirs)

puts all_dirs.map(&:size).select{|d| d <= 100000}.sum

1915606


## Part 2

โจทย์ครึ่งหลังให้เราเลือก directory ที่เล็กที่สุด ที่ยังใหญ่พอให้เมื่อลบแล้วยังเหลือขนาดที่ต้องการ

ก่อนอื่นเราก็คำนวณขนาดขั้นต่ำที่ต้องการก่อน จากนั้นก็ filter จาก directory array ของ part แรก เอาเฉพาะ directory ที่ใหญ่พอ แล้วเลือกตัวที่เล็กที่สุด

In [4]:
need = root.size - 40000000
puts all_dirs.map(&:size).select{|d| d >= need}.min

5025657
